# Setting up local AI code assist

I have been using a local llama3 LLM with ollama and the continue vs code extension with great results so far.

- install pixi
- install ollama and llms
- start ollama with llama3
- install vs code extension
- configure continue extension to use ollama


## Install pixi

We use pixi as a python manager, virtual environment, package manager and task runner all in one.
Install it below.  If on windows, make sure you have winget installed.

In [ ]:
import platform
from subprocess import call

uname = platform.uname()
if uname.system == 'Windows':
   retcode = call(["where", "pixi"])
   if retcode != 0:
      print("installing pixi")
      !winget install prefix-dev.pixi
   else:
      print("pixi is installed")
else:
   retcode = call(["which", "pixi"])
   if retcode != 0:
      print("installing pixi")
      !curl -fsSL https://pixi.sh/install.sh | bash
   else:
      print("pixi is installed")

## Install ollama and llms

We already have some pixi tasks defined, so we will run them to install the ollama runner and
the LLMs we will use for code assisstance

In [ ]:
if uname.system == "Windows":
    retcode = call(["where", "ollama"])
    if retcode!= 0:
        print("installing ollama")
        !winget install ollama
    else:
        print("ollama is installed")
else:
    retcode = call(["which", "ollama"])
    if retcode!= 0:
        print("installing ollama")
        !pixi run ollama-install
    else:
        print("ollama is installed")
!pixi run ollama-pull

## Start ollama with llama3

We need to have ollama start llama3:8b so that the vscode extension can send messages to it.  Instead of running it in a cell in the notebook, run this in your shell instead

```shell
ollama run llama3:8b
```

## Install vs code extension

Next we will install the vs code extension that allows us to hook into the LLM's we installed
through ollama.  Again, we have a pixi task to help us

In [ ]:
!pixi run vscode-ext

## Configure continue extension 

Finally, we need to configure the continue extension so that it uses the ollama runner and the LLMs we installed above.

Click on the gear icon in the bottom right corner of the view and select "Configure".  In the 
editor that pops up, enter the following:

```json
{
  "models": [
    {
      "title": "Llama3-8b",
      "model": "llama3:8b",
      "contextLength": 4096,
      "provider": "ollama"
    }
  ],
  "tabAutoCompleteModel": {
    "title": "Tab Complete Model",
    "provider": "ollama",
    "model": "starcoder2:3b"
  },
  "customCommands": [
    {
      "name": "test",
      "prompt": "{{{ input }}}\n\nWrite a comprehensive set of unit tests for the selected code. It should setup, run tests that check for correctness including important edge cases, and teardown. Ensure that the tests are complete and sophisticated. Give the tests just as chat output, don't edit any file.",
      "description": "Write unit tests for highlighted code"
    }
  ],
  "allowAnonymousTelemetry": false
}
```



## AI Code Assist examples

Here are a few examples of what the AI code assist can do for you

### Infer command from context

Here's an example of some text in a comment, and the AI determines the command that should be run.

![infer-command](./resources/complete_apt_install.png "Infer command")

### Code complete based on definitions

Here's an example where the AI knows what a data type is.  I started to type in "The participant"
and the AI predicted what code I would want to write.

![complete-on-def](./resources/complete_participant.png "Definition Completion")

### Predict based on earlier code

Here's an example where I started a new cell in the jupyter notebook.  Noticed that in
the previous cell, I had a list of steps to run.  In the next cell, based on the code
I had written for that cell, it predicted the command based on the first bullet in the
earlier cell.

![earlier-code](./resources/complete_pip_install.png "Earlier Code")

### Context aware completion

This example shows that not only did it know what code I would write, but it also knew that
the key `channel` was related to how the event was sent.  Note the suggested code was "the channel used to contact the participant".  The AI figured out that _channel_ was some kind of medium that
the message could be delivered through on its own.

![context-aware](./resources/complete_code.png  "Context Aware")